<a href="https://colab.research.google.com/github/krislars/DustTools/blob/main/Reddening_Free_Indices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reddening-Free Indices

Kristen Larson (krislars@gmail.com)

updated August 4, 2023

## Introduction

Reddening-free indices are pseudomagnitudes that are the same for both reddened and un-reddened sources.  In this demonstration, I will follow the [procedure outlined here (PDF)](http://www.astroscu.unam.mx/rmaa/RMxAC..44/PDF/RMxAC..44_VVV_catelan.pdf); see also references therein, including [Catelan et al (2011)](https://ui.adsabs.harvard.edu/abs/2011rrls.conf..145C/abstract).

A reddening-free index can be defined as

$$
m_x = m_i - \left(\frac{A_i/A_V}{A_j/A_V-A_k/A_V}\right)(m_j-m_k)
$$

where $j\neq k$.  Tip for students: Try writing this for a source with extinction and without extinction, then subtract and show that the difference is zero.

To show how indices like these can be used to identify globular clusters, I use intrinsic magnitudes from [Chen, Zhao, \& Zhao (2009)](https://ui.adsabs.harvard.edu/abs/2009ApJ...702.1336C/abstract), "The Absolute Magnitudes of Red Horizontal Branch Stars in the ugriz System", chosen primarily for ease of getting the intrinsic magnitudes into this notebook.

## Imports and installs

In [39]:
!pip install speclite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.3/892.3 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for speclite: filename=speclite-0.16-py3-none-any.whl size=366385 sha256=58da1a7b6e42293a650ac372cdb6c300050a5dd50aed3feb07e317b0f4e6723f
  Stored in directory: /root/.cache/pip/wheels/ae/53/1b/757776b1b03808f5563a0facba5402749d6493a3e0d0698b23
Successfully built speclite


In [80]:
!pip install dust_extinction

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.0/447.0 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dust_extinction: filename=dust_extinction-1.2-py3-none-any.whl size=432578 sha256=274996fe0b3e72d5b6c545ec4b01065c2b6802dfccf46999b3db0a18c196ff88
  Stored in directory: /root/.cache/pip/wheels/ff/88/1c/f994a793a0001bf0450f2147c98c28fa6ee2abfa85f3b9cf98
Successfully built dust_extinction


In [81]:
import numpy as np
import astropy.table as t
from speclite import filters as filters
from dust_extinction.parameter_averages import F19

## Get intrinsic magnitudes

I am using [Table 2](https://iopscience.iop.org/0004-637X/702/2/1336/suppdata/apj316794t2_ascii.txt?doi=10.1088/0004-637X/702/2/1336) from [Chen, Zhao, \& Zhao (2009)](https://ui.adsabs.harvard.edu/abs/2009ApJ...702.1336C/abstract) for the intrinsic magnitudes of the horizontal branch, skipping the last two anomalous clusters.

In [33]:
hb = t.Table(names=('Cluster', '(g-r)_0', 'sigmagr', 'M_u', 'sigmaM_u', 'M_g',
                    'sigmaM_g', 'M_r', 'sigmaM_r', 'M_i', 'sigmaM_i', 'M_z',
                    'sigmaM_z', 'M_V', 'M_R', 'M_I'),dtype=('str','float',
                    'float','float','float','float','float','float','float',
                    'float','float','float','float','float','float','float'))

In [34]:
hb.add_row(['Pal 3', 0.25, 0.08, 1.71, 0.25, 0.60, 0.13, 0.28, 0.08, 0.28, 0.08, 0.28, 0.10, 0.40, 0.12, -0.09])
hb.add_row(['NGC 7006', 0.31, 0.13, 1.80, 0.15, 0.60, 0.30, 0.37, 0.22, 0.27, 0.26, 0.27, 0.26, 0.45, 0.18, -0.13])
hb.add_row(['M3', 0.28, 0.12, 1.80, 0.22, 0.64, 0.21, 0.40, 0.30, 0.31, 0.30, 0.31, 0.32, 0.49, 0.22, -0.09])
hb.add_row(['Pal 14', 0.40, 0.12, 1.91, 0.27, 0.77, 0.06, 0.35, 0.15, 0.15, 0.16, 0.07, 0.22, 0.51, 0.14, -0.28])
hb.add_row(['Pal 4', 0.40, 0.09, 1.88, 0.35, 0.60, 0.13, 0.20, 0.08, 0.17, 0.08, -0.05, 0.15, 0.36, 0.04, -0.21])
hb.add_row(['Pal 5', 0.29, 0.08, 1.71, 0.07, 0.62, 0.24, 0.25, 0.19, 0.25, 0.19, 0.14, 0.27, 0.39, 0.09, -0.12])
hb.add_row(['M5', 0.27, 0.15, 1.84, 0.14, 0.67, 0.17, 0.39, 0.17, 0.30,0.20, 0.30, 0.23, 0.50, 0.21, -0.10])

In [35]:
hb

Cluster,(g-r)_0,sigmagr,M_u,sigmaM_u,M_g,sigmaM_g,M_r,sigmaM_r,M_i,sigmaM_i,M_z,sigmaM_z,M_V,M_R,M_I
str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
Pal 3,0.25,0.08,1.71,0.25,0.6,0.13,0.28,0.08,0.28,0.08,0.28,0.1,0.4,0.12,-0.09
NGC 7006,0.31,0.13,1.8,0.15,0.6,0.3,0.37,0.22,0.27,0.26,0.27,0.26,0.45,0.18,-0.13
M3,0.28,0.12,1.8,0.22,0.64,0.21,0.4,0.3,0.31,0.3,0.31,0.32,0.49,0.22,-0.09
Pal 14,0.4,0.12,1.91,0.27,0.77,0.06,0.35,0.15,0.15,0.16,0.07,0.22,0.51,0.14,-0.28
Pal 4,0.4,0.09,1.88,0.35,0.6,0.13,0.2,0.08,0.17,0.08,-0.05,0.15,0.36,0.04,-0.21
Pal 5,0.29,0.08,1.71,0.07,0.62,0.24,0.25,0.19,0.25,0.19,0.14,0.27,0.39,0.09,-0.12
M5,0.27,0.15,1.84,0.14,0.67,0.17,0.39,0.17,0.3,0.2,0.3,0.23,0.5,0.21,-0.1


I do a simple weighted average over all clusters.

In [97]:
m0 = np.array([np.average(hb['M_u'],weights=hb['sigmaM_u']),
               np.average(hb['M_g'],weights=hb['sigmaM_g']),
               np.average(hb['M_r'],weights=hb['sigmaM_r']),
               np.average(hb['M_i'],weights=hb['sigmaM_i']),
               np.average(hb['M_z'],weights=hb['sigmaM_z'])])
print('M0_band=',m0)

M0_band= [1.8237931  0.62846774 0.3412605  0.2603937  0.20135484]


## Get filter nominal effective wavelengths

I am using ``speclite`` to get the effective wavelengths of the DECam filters to look up the extinction law values.  This package can also be useful for getting synthetic photometry of a model spectrum using a spectroscopic extinction law.

In [43]:
decam = filters.load_filters('decam2014-*')

In [108]:
lam_eff=decam.effective_wavelengths
lam_eff

<Quantity [3964.66971417, 4890.03670428, 6469.62203811, 7847.78249813,
           9196.46396394, 9892.9684439 ] Angstrom>

## Get extinction curve

The ``dust_extinction`` package has many extinction laws in it. I somewhat arbitrarily choose the Fitzpatrick 2019 extinction law.  Because the intrinsic colors were dereddened with a particular extinction law, there is a circular problem here that probably should be investigated more carefully.

Note that simply looking up the extinction at the effective wavelength of a filter passband leads to some complicated uncertainties, espeically for sources at extreme colors.  Better would be to use a spectroscopic extinction law with a model spectrum and then perform synthetic photometry to get the extinction in magnitudes.  

In [82]:
ext = F19(Rv=3.1)

In [107]:
ext(lam_eff)

array([1.44284981, 1.18151869, 0.83262709, 0.62999379, 0.48488961,
       0.43959079])

## Calculate indices

The next question is what filters to choose.  

I am only going to consider colors between adjacent passbands for the color term.  Colors across large wavelength ranges will have larger errors in the reddening term.  In other words, I choose colors where $|j-k|$=1.  In the case of $ugriz$, there are 4+3+2+1=10 different colors.

In [76]:
bands=('u', 'g', 'r', 'i', 'z')
for j in range(5):
    for k in range(j+1,5):
        print(bands[j],'-',bands[k])

u - g
u - r
u - i
u - z
g - r
g - i
g - z
r - i
r - z
i - z


With this restriction on color, there are 50 different choices for reddening-free indices.  Here we calculate them all.

In [95]:
mx=t.Table(names=('i','j-k','m_x'),dtype=('str','str','float'))
for i in range(5):
    for j in range(5):
        for k in range(j+1,5):
            mx.add_row([bands[i],bands[j]+'-'+bands[k],
                        np.round(m0[i]-(ext(lam_eff[i])/(ext(lam_eff[j])-ext(lam_eff[k])))*(m0[j]-m0[k]),3)])
mx.pprint(max_lines=100)

 i  j-k  m_x  
--- --- ------
  u u-g -4.776
  u u-r -1.682
  u u-i -0.951
  u u-z  -0.62
  u g-r  0.636
  u g-i  0.861
  u g-z  0.939
  u r-i  1.248
  u r-z  1.243
  u i-z  1.237
  g u-g -4.776
  g u-r -2.242
  g u-i -1.644
  g u-z -1.373
  g g-r -0.344
  g g-i  -0.16
  g g-z -0.096
  g r-i  0.157
  g r-z  0.153
  g i-z  0.148
  r u-g -3.467
  r u-r -1.682
  r u-i  -1.26
  r u-z -1.069
  r g-r -0.344
  r g-i -0.214
  r g-z -0.169
  r r-i  0.009
  r r-z  0.006
  r i-z  0.002
  i u-g -2.621
  i u-r  -1.27
  i u-i -0.951
  i u-z -0.807
  i g-r -0.258
  i g-i  -0.16
  i g-z -0.126
  i r-i  0.009
  i r-z  0.007
  i i-z  0.004
  z u-g -2.017
  z u-r -0.977
  z u-i -0.731
  z u-z  -0.62
  z g-r -0.198
  z g-i -0.122
  z g-z -0.096
  z r-i  0.008
  z r-z  0.006
  z i-z  0.004


The next obvious step is a more careful accounting of uncertainty.

One way to check these indices would be to calculate them for these same clusters before extinction corrections were applied to see if the values come out the same.  In other words, check to make sure they are indeed reddening free.